# Predicting Batting statistics

## Goal
The main goal of this notebook is to understand machine learning.
Using baseball statistics to try and predict a batters offensive performance for the comming year based on past years performance.

### Things to achieve
- Calculate wOBA
- Group records by player and year.
- Find outliers (Injury, Team change, etc.)
- Project wOBA using a players past wOBA.

## Load Data from csv file

In [1]:
import pandas as pd

raw_data = pd.read_csv('data/Batting.csv')

## Baseball statistic to quantify offensive production - wOBA

wOBA is a sabermetric statistic used to measure a hitter's overall offensive contributions per plate appearance, assigning more accurate weights to different offensive outcomes.
General Formula:
$$
wOBA = \frac{(wBB \times BB) + (wHBP \times HBP) + (w1B \times 1B) + (w2B \times 2B) + (w3B \times 3B) + (wHR \times HR)}{AB + BB - IBB + SF + HBP}
$$
Where:
- \( wBB, wHBP, w1B, w2B, w3B, wHR \) are weights that change slightly every year.
- \( BB \) = Base on balls (walks)
- \( IBB \) = Intentional walks
- \( HBP \) = Hit by pitch
- \( AB \) = At bats
- \( SF \) = Sacrifice flies
- \( 1B \) = Singles


### Calculate Singles

Singles are not included in the dataset.
To represent this we take the total hits('H') and remove doubles('2B'), triples('3B') and homeruns('HR')

In [2]:
work_df = raw_data.copy()
work_df['1B'] = work_df['H'] - work_df['2B'] - work_df['3B'] - work_df['HR']
work_df['1B'] = work_df['1B'].fillna(0)
work_df['1B'] = work_df['1B'].astype(int)

## wOBA weights
During the prototyping phases static weights are used.

To improve accuracy yearly weights can be calculate form the dataset. 

In [3]:
wBB = 0.69
wHBP = 0.72
w1B = 0.88
w2B = 1.247
w3B = 1.578
wHR = 2.031

## Cleaned dataset

Remove records with empty empty values in required columns

In [14]:
required_columns = ['BB', 'IBB', 'HBP', '1B', '2B', '3B', 'HR', 'AB', 'SF']
cleaned_df = work_df.dropna(subset=required_columns)

## Save to CSV

Save cleaned dataframe (cleaned_df) to cleaned.csv 

In [16]:
cleaned_df.to_csv("data/cleaned.csv", index=False)

## Calculate wOBA Numerator & Denominator

In [17]:
# Numerator
cleaned_df['wOBA_num'] = (
    wBB * (cleaned_df['BB'] - cleaned_df['IBB']) +
    wHBP * cleaned_df['HBP'] +
    w1B * cleaned_df['1B'] +
    w2B * cleaned_df['2B'] +
    w3B * cleaned_df['3B'] +
    wHR * cleaned_df['HR']
)

#Denominator
cleaned_df['wOBA_deno'] = cleaned_df['AB'] + cleaned_df['BB'] - cleaned_df['IBB'] + cleaned_df['HBP'] + cleaned_df['SF']

### Calculate wOBA

In [18]:
cleaned_df['wOBA'] = cleaned_df['wOBA_num'] / cleaned_df['wOBA_deno']
cleaned_df['wOBA'] = cleaned_df['wOBA'].round(3)

In [19]:
cleaned_df.to_csv("data/woba.csv", index=False)